# 🦌 ELK XELK Transformer Layout Options 🤖

The `XELKTypedLayout` generates a nested dictionary that controls various
[Layout Options](https://www.eclipse.org/elk/reference/options.html) that can be passed
to Elk. Currently only a subset of Layout Options are implemented and not all
interdepenencies are taken into account.

In [ ]:
import json
import pathlib

import importnb
import ipywidgets
import networkx
import traitlets
from IPython.display import JSON, display

import ipyelk
import ipyelk.nx
import ipyelk.tools
from ipyelk.diagram import layout_options

In [ ]:
with importnb.Notebook():
    from __02_Transformer import (
        a_hierarchical_elk_example,
        a_collapsible_elk_example,
        a_flat_elk_json_example,
    )

In [ ]:
if __name__ == "__main__":
    xelk_typed_layout = ipyelk.nx.XELKTypedLayout()

The output `value` of the widget is a nested dictionary. Top level keys are nodes in
networkx. The keys of the next level are `ElkGraphElements` e.g. (`ElkNode`, `ElkLabel`,
...). The last layer are actual Elk layout options key value pairs.

The `XELK` transformer will look to this structure to identify layout options based on
Networkx node and which ElkGraphElements generated from the node.

In [ ]:
if __name__ == "__main__":
    display(xelk_typed_layout, JSON(xelk_typed_layout.value))

## Flat structure

A `networkx.MultiDigraph` can be used to create a flat graph.

> _TODO: There should be an option to specify if ports should be created or only connect
> edges between the nodes_

In [ ]:
def a_flat_elk_json_example_with_layouts(layouts=None):
    layouts = layouts or ipyelk.nx.XELKTypedLayout()
    elk, xelk = a_flat_elk_json_example()
    xelk.layouts = layouts.value

    def _element_type_opt_change(change):
        xelk.layouts = xelk_typed_layout.value
        xelk.refresh()

    layouts.observe(_element_type_opt_change, "value")
    box = ipywidgets.HBox([elk, layouts])
    return box, layouts, xelk, elk

In [ ]:
if __name__ == "__main__":
    display(a_flat_elk_json_example_with_layouts(xelk_typed_layout)[0])

## Hierarchical Diagram with Ports

In [ ]:
def a_hierarchical_elk_example_with_layouts(layouts=None):
    layouts = layouts or ipyelk.nx.XELKTypedLayout()
    box, elk, xelk = a_collapsible_elk_example()
    box.layout.flex = "1"
    xelk.layouts = layouts.value

    def _type_opt_change(change):
        xelk.layouts = layouts.value
        xelk.refresh()

    layouts.observe(_type_opt_change, "value")

    box2 = ipywidgets.HBox([box, layouts], layout=dict(max_height="60vh"))
    return box2, elk, xelk, layouts

In [ ]:
if __name__ == "__main__":
    display(a_hierarchical_elk_example_with_layouts()[0])

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).